In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


In [3]:
# read table

dataPayment = './Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

In [ ]:
# specify table column included

payroll = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Job Title', 'Base Pay', 'Position ID', 'Employee Identifier']]

# add Working Year Column
payroll["Working Year"] = _work_year

In [55]:
# YANG INI JANGAN DIJALANKAN DULU YA GESSS :'

arr = []

try:
    with open('./cache/ids.txt', "r") as f:
        for _id in f:
            arr.append(int(_id))
except:
    if not os.path.exists('./cache'):
        os.mkdir('./cache')
        
    _index = payroll['Job Code'].unique()
    for _id in _index:
        counts = len(payroll.groupby(['Job Code']).get_group(_id))
        if counts > 5000:
            arr.append(str(_id))
    with open('cache/ids.txt', 'w') as f:
        for _id in arr:
            f.write('%s\n' % _id) 
finally:
   f.close()

# :return : <List> arr : list of unique job id

In [16]:
# Group Job Code 1172 training dataset <<< PAKAI INI NGGIH

# group_1172 = payroll.groupby(['Job Code']).get_group(1172)

# _sorted = group_1172.sort_values('Employee Identifier')
# fiscal = pd.pivot_table(_sorted, values='Base Pay', index=['Fiscal Period'], columns='Employee Identifier')

# fiscal = fiscal.diff()

In [ ]:
# cleaned display table

# fiscal.fillna(0, inplace=True)
# fiscal = fiscal.drop(2016.00)

# fiscal

In [65]:
# query job code indexing

lb = pd.DataFrame()
for val in arr:
    new_df = payroll[payroll['Job Code'] == val]
    lb = pd.concat([lb, new_df], ignore_index=True)
lb

,Fiscal Year,Fiscal Period,Job Code,Job Title,Base Pay,Position ID,Employee Identifier,Working Year
0,2016,2016.00,1172,Assistant State's Attorney,20088.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.00
1,2016,2016.25,1172,Assistant State's Attorney,23436.00,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.25
2,2016,2016.50,1172,Assistant State's Attorney,20422.82,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.50
3,2016,2016.75,1172,Assistant State's Attorney,23904.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,11.75
4,2017,2017.00,1172,Assistant State's Attorney,20745.80,9510200,6ac7ba3e-d286-44f5-87a0-191dc415e23c,12.00
...,...,...,...,...,...,...,...,...
66577,2018,2018.00,907,Clerk V,12040.53,9520059,fb5ae7f8-115f-4485-84c0-ec2157fd992d,23.00
66578,2018,2018.00,907,Clerk V,12033.20,1700434,72488c49-8102-465e-905e-d7f9bf9d42a4,30.00
66579,2018,2018.00,907,Clerk V,11989.79,9512179,76a191a5-67e3-40a9-bb2f-d7339cb8ca22,24.00
66580,2018,2018.25,907,Clerk V,13731.34,9516739,bae08550-6349-4370-befd-128e7a0bb5e1,21.25


In [ ]:
X = lb[['Job Code','Working Year','Position ID','Fiscal Period', 'Fiscal Year']]
Y = lb['Base Pay']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size =0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RidgeCV
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

In [69]:
y_pred = rf.predict(x_test)

In [70]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)
# accuracy_score(y_test, y_pred)

2452.384201705992